In [11]:
%matplotlib inline

In [12]:
%load_ext autoreload
%autoreload 1
%aimport numpy_api_stats

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import os
os.environ['GOOGLE_CRED_DIR']=os.path.join(os.environ['HOME'],'creds')
os.environ['GOOGLE_APPLICATION_CREDENTIALS']=os.path.join(os.environ['GOOGLE_CRED_DIR'],'apt-footing-235018-aeb185ac9e31.json')

In [15]:
import numpy_api_stats as ns


#### Investigate bigQuery public github dataset contents using bqhelper

In [ ]:
#from google.cloud import bigquery
github_repos = ns.BigQueryHelper(active_project= "bigquery-public-data", 
                                       dataset_name = "github_repos")

In [ ]:
github_repos.list_tables()

In [ ]:
api_results = ns.BigQueryHelper(active_project= "apt-footing-235018", 
                                       dataset_name = "NumpyAPI")
api_results.list_tables()

In [ ]:
github_repos.table_schema('files')

### Construct API datasets

In [16]:
import numpy as np
import pandas as pd

api = [(x, type(np.__getattribute__(x))) for x in dir(np) if not x.startswith('__')]
things = set([ x[1].__name__ for x in api])
names = set([ x[0] for x in api])
df_api = pd.DataFrame(api,columns=['name','type'])

array_type = df_api[df_api.name=='array']
things_of_array_type = df_api[df_api.type == array_type.type.values[0]]
#things_of_array_type

In [43]:
df_api[df_api.name=='linalg']

,name,type
339,linalg,<class 'module'>


In [17]:
GoogleSheet = pd.read_csv('NumPy API - Sheet2.csv')
names_from_sheet = GoogleSheet['NumPy API']

In [18]:
valid_names = set(names_from_sheet)-set(['Functions','bench',np.nan])

In [19]:
new_names = set(names)-set(names_from_sheet)

In [34]:
df_valid_from_sheet = df_api.set_index('name').loc[valid_names]
df_valid_from_sheet = df_valid_from_sheet.reset_index()
df_valid_from_sheet['type'] = df_valid_from_sheet.type.apply(lambda x: x.__name__)
df_valid_from_sheet.head()

,name,type
0,bytes0,type
1,ndim,function
2,meshgrid,function
3,generic,type
4,singlecomplex,type


#### How many of each type in the api are represented in the Google Sheet?

In [38]:
df_valid_from_sheet[['type','name']].groupby(['type']).count().sort_values('name')

,name
type,
CClass,1
NoneType,1
PytestTester,1
RClass,1
bool,1
int,1
IndexExpression,2
_typedict,2
str,2


#### Of which, the following have not been implemented

In [40]:
df_valid_from_sheet.set_index('type').loc[['CClass','NoneType','PytestTester','RClass','bool','IndexExpression','_typedict','str','nd_grid','_Feature','dict']]

,name
type,
CClass,c_
NoneType,newaxis
PytestTester,test
RClass,r_
bool,little_endian
IndexExpression,s_
IndexExpression,index_exp
_typedict,cast
_typedict,nbytes


#### Query Constructed for API functions..  

In [44]:
# Selecting just a few for demonstration, Query price is independent of # of regex though, so it is better to actually run
# with all at once.

funs = ns.select(api,'function')
mods = ns.select(api,'module')
ints = ns.select(api,'int')
floats = ns.select(api,'float')
ufuncs = ns.select(api,'ufunc')
api_list = funs[:2]+mods[:2]+ints[:2]+floats[:2]+ufuncs[:2]

In [50]:
# Use all to build full query
api_list=ns.build_api_list(api)
len(api_list)

91

In [52]:
import numpy_api_stats as ns
#apq = ns.API_QUERY_FACTORY(api_list=api_list)
apq = ns.API_QUERY_FACTORY()
print(apq.query)

SELECT
REGEXP_MATCH(c.content,'(import\s+numpy\.|from numpy\._distributor_init\s+import\s+[a-zA-Z0-9_]+)' ) AS numpy_module__distributor_init,
REGEXP_MATCH(c.content,'(import\s+numpy\.|from numpy\._globals\s+import\s+[a-zA-Z0-9_]+)' ) AS numpy_module__globals,
REGEXP_MATCH(c.content,'(import\s+numpy\.|from numpy\._import_tools\s+import\s+[a-zA-Z0-9_]+)' ) AS numpy_module__import_tools,
REGEXP_MATCH(c.content,'(import\s+numpy\.|from numpy\._mat\s+import\s+[a-zA-Z0-9_]+)' ) AS numpy_module__mat,
REGEXP_MATCH(c.content,'(import\s+numpy\.|from numpy\._mklinit\s+import\s+[a-zA-Z0-9_]+)' ) AS numpy_module__mklinit,
REGEXP_MATCH(c.content,'(import\s+numpy\.|from numpy\.add_newdocs\s+import\s+[a-zA-Z0-9_]+)' ) AS numpy_module_add_newdocs,
REGEXP_MATCH(c.content,'(import\s+numpy\.|from numpy\.char\s+import\s+[a-zA-Z0-9_]+)' ) AS numpy_module_char,
REGEXP_MATCH(c.content,'(import\s+numpy\.|from numpy\.compat\s+import\s+[a-zA-Z0-9_]+)' ) AS numpy_module_compat,
REGEXP_MATCH(c.content,'(import\s+n

### AFter copying and pasting the above query into BigQuery and saving as sample_results... 

#####Unfortunately this doesn't work https://github.com/dask/dask/issues/3121 so we manually export results.

In [ ]:
#Pull data from bigQuery
#r = api_results.query_to_pandas_safe('SELECT * FROM `apt-footing-235018.NumpyAPI.sample_results`')
#r

In [ ]:
import pandas as pd

In [ ]:
ns.plot_results(pd.read_csv('results.csv'))